In [10]:
# https://api.launchpad.net/1.0/

from launchpadlib.launchpad import Launchpad

lp = Launchpad.login_anonymously("ubuntu-build-list", "production")
ubuntu = lp.distributions["ubuntu"]
archive = ubuntu.getArchive(name="primary")
series = list(ubuntu.series)

In [26]:
series_dict = {
    s.version: s
    for s in series
    if (16, 4) <= tuple(int(part) for part in s.version.split(".")) <= (24, 4)
}
series_dict.keys()

dict_keys(['24.04', '23.10', '23.04', '22.10', '22.04', '21.10', '21.04', '20.10', '20.04', '19.10', '19.04', '18.10', '18.04', '17.10', '17.04', '16.10', '16.04'])

In [81]:
import re
from pathlib import Path


def get_ddeb(
    distro_series,
    source_name: str = "linux",
    pocket: str = "Release",
    arch: str = None,
    flavor: str = None,
):
    published_sources = archive.getPublishedSources(
        exact_match=True,
        source_name=source_name,
        distro_series=distro_series,
        pocket=pocket,
        order_by_date=True,
    )

    if len(published_sources) == 0:
        raise ValueError(f"No published sources for {source_name} in {distro_series}")

    versions = [
        tuple(int(part) for part in re.split(r"-|\.|\+", s.source_package_version))
        for s in published_sources
    ]
    if max(versions) != versions[0]:
        raise ValueError(f"The latest version is not the first: {versions}")

    source = published_sources[0]
    web_link = f"https://launchpad.net/ubuntu/+source/{source_name}/{source.source_package_version}"

    urls = []
    for url in source.binaryFileUrls():
        path = Path(url)
        if path.suffix != ".ddeb":
            continue
        if not path.stem.startswith("linux-image"):
            continue
        if arch is not None and not path.stem.endswith(arch):
            continue
        if flavor is not None and flavor not in path.stem:
            continue
        urls.append(url)
    return web_link, urls

In [47]:
# https://launchpad.net/ubuntu/+source/linux/<version>

for s in series_dict.values():
    web_link, urls = get_ddeb(s, arch="amd64", flavor="generic")
    assert len(urls) == 1, urls
    print(s.version, urls[0])

24.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.8.0-22-generic-dbgsym_6.8.0-22.22_amd64.ddeb
23.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.5.0-9-generic-dbgsym_6.5.0-9.9_amd64.ddeb
23.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-6.2.0-20-generic-dbgsym_6.2.0-20.20_amd64.ddeb
22.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.19.0-21-generic-dbgsym_5.19.0-21.21_amd64.ddeb
22.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.15.0-25-generic-dbgsym_5.15.0-25.25_amd64.ddeb
21.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.13.0-19-generic-dbgsym_5.13.0-19.19_amd64.ddeb
21.04 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.11.0-16-generic-dbgsym_5.11.0-16.17_amd64.ddeb
20.10 https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.8.0-25-generic-dbgsym_5.

In [75]:
web_link, urls = get_ddeb(series_dict["20.04"])
print(web_link)
print()
print("\n".join(urls))

https://launchpad.net/ubuntu/+source/linux/5.4.0-26.30

https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_s390x.ddeb
https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_ppc64el.ddeb
https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_amd64.ddeb
https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-lowlatency-dbgsym_5.4.0-26.30_amd64.ddeb
https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_arm64.ddeb
https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-26-generic-dbgsym_5.4.0-26.30_armhf.ddeb
https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-26-generic-lpae-dbgsym_5.4.0-26.30_armhf.ddeb


In [86]:
for source_name in [
    "linux",
    "linux-azure",
    "linux-aws",
    "linux-gcp",
    "linux-oracle",
    # "linux-kvm",  # No debug symbols
    # "linux-raspi", # Only for arm64
    # "linux-laptop", # Only for arm64 with newer kernels
    # "linux-nvidia", # Only for newer kernels
]:
    web_link, urls = get_ddeb(
        series_dict["20.04"],
        source_name=source_name,
        arch="amd64",
    )
    print(f"{web_link:64}{', '.join(urls)}")

https://launchpad.net/ubuntu/+source/linux/5.4.0-26.30          https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-generic-dbgsym_5.4.0-26.30_amd64.ddeb, https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-26-lowlatency-dbgsym_5.4.0-26.30_amd64.ddeb
https://launchpad.net/ubuntu/+source/linux-azure/5.4.0-1010.10  https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-1010-azure-dbgsym_5.4.0-1010.10_amd64.ddeb
https://launchpad.net/ubuntu/+source/linux-aws/5.4.0-1009.9     https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-5.4.0-1009-aws-dbgsym_5.4.0-1009.9_amd64.ddeb
https://launchpad.net/ubuntu/+source/linux-gcp/5.4.0-1009.9     https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-1009-gcp-dbgsym_5.4.0-1009.9_amd64.ddeb
https://launchpad.net/ubuntu/+source/linux-oracle/5.4.0-1009.9  https://launchpad.net/ubuntu/+archive/primary/+files/linux-image-unsigned-5.4.0-